<h3> Imports </h3>

In [1]:
import numpy as np
import pandas as pd
import ebooklib
import re
import nltk
from nltk.util import ngrams
from ebooklib import epub
from bs4 import BeautifulSoup
from spacy.lang.en import English

<h3>Reading the epub file : </h3>

In [2]:
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

In [3]:
blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script',   ]

In [4]:
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

In [5]:
def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

In [6]:
def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

In [7]:
#epub file as a list
out=epub2text('The-Art-of-Digital-Marketing-The-Definitive-Guide-to-Creating-Strategic-Targeted-and-Measurable-Online-Campaigns.epub')

In [8]:
out

['\n \n \n \n \n ',
 '\n Contents \n \n Cover \n Title Page \n Copyright \n Preface \n Chapter 1: An Introduction to Digital Marketing \n \n What Makes This Book Different? \n Start with the Customer and Work Backward \n What Are the 3i Principles? \n \n \n Chapter 2: Search Engine Optimization \n \n An Introduction \n Search Engine Result Pages: Positioning \n Search Behavior \n Stage 1: Goals \n Stage 2: On-Page Optimization \n Stage 3: Off-Page Optimization \n Stage 4: Analyze \n So, What Have You Learned in This Chapter? \n \n \n Chapter 3: Pay Per Click \n \n An Introduction \n Stage 1: Goals \n Stage 2: Setup \n Stage 3: Manage \n Stage 4: Analyze \n So, What Have You Learned in This Chapter? \n \n \n Chapter 4: Digital Display Advertising \n \n An Introduction \n Display Advertising: An Industry Overview \n Stage 1: Define \n Stage 2: Format \n Stage 3: Configure \n Stage 4: Analyze \n So, What Have You Learned in This Chapter? \n \n \n Chapter 5: Email Marketing \n \n An Introd

<h3> Data Munging </h3>

In [9]:
#creating a string out of the text
text_epub = ''.join(out)

In [10]:
text_epub

"\n \n \n \n \n \n Contents \n \n Cover \n Title Page \n Copyright \n Preface \n Chapter 1: An Introduction to Digital Marketing \n \n What Makes This Book Different? \n Start with the Customer and Work Backward \n What Are the 3i Principles? \n \n \n Chapter 2: Search Engine Optimization \n \n An Introduction \n Search Engine Result Pages: Positioning \n Search Behavior \n Stage 1: Goals \n Stage 2: On-Page Optimization \n Stage 3: Off-Page Optimization \n Stage 4: Analyze \n So, What Have You Learned in This Chapter? \n \n \n Chapter 3: Pay Per Click \n \n An Introduction \n Stage 1: Goals \n Stage 2: Setup \n Stage 3: Manage \n Stage 4: Analyze \n So, What Have You Learned in This Chapter? \n \n \n Chapter 4: Digital Display Advertising \n \n An Introduction \n Display Advertising: An Industry Overview \n Stage 1: Define \n Stage 2: Format \n Stage 3: Configure \n Stage 4: Analyze \n So, What Have You Learned in This Chapter? \n \n \n Chapter 5: Email Marketing \n \n An Introduction

In [11]:
#using spacy
nlp = English()
doc = nlp(text_epub)

In [26]:
#tokenizing
def tokenize_whitespace(text_epub):
    tokens = []
    for l in text_epub:
        tokens.extend(l.split())
    return tokens


tokens = tokenize_whitespace(text_epub)
len(tokens)

551576

In [18]:
#removing newlines
tokens = [token.strip('\n') for token in tokens]

In [19]:
#removing numbers
def filter_numbers(string):
    return re.sub(r'\d+', '', string)

tokens = [filter_numbers(token) for token in tokens]

In [20]:
#removing punctuation
def filter_punct(string):
    return re.sub(r'[^\w\s]', '', string)

tokens = [filter_punct(token) for token in tokens]

In [21]:
#removing empty space
tokens = [token for token in tokens if token]

In [22]:
len(tokens)

529966